In [7]:
import time
import numpy as np
import pandas as pd
import datetime

from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


# Controlling the webdriver  
Firstly, we want to access the webpage doctolib. To do that we have to create a connector with Selenium and the launch the driver

In [8]:
tempsAttente = 3    #temps d'attente en secondes avant qu'il y a un timeout de Webdriverwait.until()

class Driver:

    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument("start-maximized")
        options.add_argument("--disable-blink-features")
        options.add_argument("disable-popup-blocking")
        options.add_argument("disable-notifications")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        options.add_argument("--incognito")
        # prefs = {"profile.default_content_settings.geolocation" : "2"}
        # options.add_experimental_option("prefs",prefs)
        self.driver = webdriver.Chrome(ChromeDriverManager().install())


    #Lancer google chrome avec la page doctolib

    def connect(self, url):
        self.driver.get(url)
        # TabsInteraction.TabsCreation(self.driver, "https://www.doctolib.fr", "")
        # TabsInteraction.switchToTab(self.driver, 0)
        print("Connecting to", url)
        time.sleep(1)


    #for now we get only the first doctors that appear in the page
    def getDoctors(self, specialization, city):

        doctors = []

        try :
            #writing the specialization
            WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]')))
            self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').clear()
            self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').send_keys(specialization)
            time.sleep(1)

            #writing the city
            WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]')))
            self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').clear()
            self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').send_keys(city)
            time.sleep(1)
            #We need to enter the city to register it
            self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').send_keys(Keys.ENTER)

            WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//span[@class="searchbar-submit-button-label dl-text-transform-none"]')))
            self.driver.find_element(By.XPATH, '//span[@class="searchbar-submit-button-label dl-text-transform-none"]').click()

            print('Searching results for :', specialization, 'in', city, 'at', datetime.datetime.now())
            time.sleep(2)

            WebDriverWait(self.driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//h4[@class="dl-text dl-text-body dl-text-bold dl-text-s dl-text-neutral-150 total-number-of-results dl-margin-r-8"]')))
            #number of doctors in city
            nbr_result = self.driver.find_element(By.XPATH, '//h4[@class="dl-text dl-text-body dl-text-bold dl-text-s dl-text-neutral-150 total-number-of-results dl-margin-r-8"]').text.split(" ")[0]
            #nbr of doctors we want
            nbr = int(int(nbr_result)*0.2)
            #nbr of pages
            pages = nbr//20
            # print(pages)

            for _ in range(pages + 1):
                
                self.driver.execute_script("window.scrollTo(0, 6300);")
                
                WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-search-result"]')))
                docs = self.driver.find_elements(By.XPATH, '//div[@class="dl-search-result"]')
                
                for doctor in docs:
                    #first we get the "easy" features
                    name = doctor.find_element(By.XPATH, './/h3[@data-test="dl-search-result-name"]').text
                    city = city
                    specialization = specialization

                    #the the harder ones
                    try:
                        WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, './/div[@class="Tappable-inactive availabilities-slot"]')))
                        first_dispo = doctor.find_element(By.XPATH, './/div[@class="Tappable-inactive availabilities-slot"]').get_attribute('aria-label')
                    except:
                        try:
                            WebDriverWait(self.driver, 1).until(EC.visibility_of_element_located((By.XPATH, './/span[@class="dl-button-label"]')))
                            first_dispo = doctor.find_elements(By.XPATH, './/span[@class="dl-button-label"]')[1].text.split(" ")[3:5]
                        except:
                            try:
                                WebDriverWait(self.driver, 1).until(EC.visibility_of_element_located((By.XPATH, './/div[@class="dl-text dl-text-body dl-text-bold dl-text-s dl-text-neutral-150"]')))
                                first_dispo = 'pas de disponibilité'
                            except:
                                first_dispo = 'null'    #means that we didn't manage to find the date
                            

                    print(name, first_dispo)
                    fee = ""
                    doctors.append(Doctor(name, city, specialization, first_dispo, fee))

                try:
                    WebDriverWait(self.driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//a[text()='Suivant']")))
                    self.driver.find_element(By.XPATH, "//a[text()='Suivant']").click()
                except:
                    pass
        
        except :
            print('Could not get the doctors for ' + specialization + ' in ' + city)

        return doctors


    def getFees(self, name):
        self.driver.find_element(By.XPATH, '//div[@class="logo-doctolib logo-doctolib-white"]').click()

        #writing the doctor name
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]')))
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').clear()
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').send_keys(name)
        #We need to enter the city to register it
        #TODO : add the city
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input searchbar-query-input-opened"]')))
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input searchbar-query-input-opened"]').send_keys(Keys.ENTER)

        try:
            WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-profile-fee-tag"]')))
            fee = self.driver.find_element(By.XPATH, '//div[@class="dl-profile-fee-tag"]').text
            fee = str(int(fee.split(" ")[0]))
        except:
            try :
                # WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, "//div[contains(text(), 'Conventionn')]")))
                # fee = self.driver.find_element(By.XPATH, "//div[contains(text(), 'Conventionn')]").text
                WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-profile-text"]')))
                fee = self.driver.find_element(By.XPATH, '//div[@class="dl-profile-text"]').text
                WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, "//a[text()='Suivant']")))
                self.driver.find_element(By.XPATH, "//a[text()='Suivant']").click()
            except:
                fee = 'null'        #if the fee tag cannot be found it means that it is conventionné

        return fee
            
        


We created a Doctor class to manipulate more easily the doctors features

In [9]:
class Doctor():
    def __init__(self, name, city, specialization, first_dispo, fee):
        self.name = name
        self.city = city
        self.specialization = specialization
        self.first_dispo = first_dispo
        self.fee = fee

    def features(self):
        return(self.name, self.city, self.specialization, self.first_dispo, self.fee)

These are the cities and specializations we looked at first :  
Lyon and Paris are bug cities and Sancerre and Confolens are small ones, we did this in order to compare fees and disponibilites at first glance.

In [10]:
# cities = ['Poitiers', 'Rochefourchat', 'Majastres', 'Aulan', 'Rouen', 'Lorient', 'Niort', 'Valence', 'Quimper', 'Paris', 'Lyon', 'Marseille', 'Toulouse', 'Nice', 'Nantes', 'Montpellier', 'Strasbourg', 'Bordeaux', 'Lille', 'Poitiers', 'Dunkerque', 'Pau', 'Mulhouse', 'Lorient', 'Le Havre', 'Annecy', 'Troyes', 'La Rochelle', 'Amiens']
# specializations = ['Medecin generaliste', 'Chirurgien-dentiste', 'Ophtalmologue', 'Dermatologue et venerologue', 'Cardiologue', 'Rhumatologue']

cities = ['Toulouse', 'Nice', 'Nantes', 'Montpellier', 'Strasbourg', 'Bordeaux', 'Lille', 'Poitiers', 'Dunkerque', 'Pau', 'Mulhouse', 'Lorient', 'Le Havre', 'Annecy', 'Troyes', 'La Rochelle', 'Amiens']
specializations = ['Medecin generaliste', 'Chirurgien-dentiste', 'Ophtalmologue', 'Dermatologue et venerologue', 'Cardiologue', 'Rhumatologue']


In [11]:
connector = Driver()
connector.connect("https://www.doctolib.fr")
doctors = []

#Get all the doctors names, cities, specializations and their next possible reservation

for city in cities:
    for spec in specializations:
        doctors = doctors + connector.getDoctors(spec, city)
        docs = []
        for doc in doctors:
            docs.append(doc.features())
        pd.DataFrame(docs, columns=['Name', 'City', 'Specialization', 'Waiting Time', 'Fees']).to_csv("doctors2.csv")

/var/folders/bm/t8hxv1ms6s30y8czfw0pmfk00000gn/T/ipykernel_2201/2896691625.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())


Connecting to https://www.doctolib.fr
Searching results for : Medecin generaliste in Toulouse at 2023-01-12 14:22:49.559969
Dr Laetitia CAZES pas de disponibilité
Pharmacie Sorbette ven. 13 janv. 13:00
Dr Lucie Verret pas de disponibilité
Dr Patrick BOUDAL pas de disponibilité
Dr Jean-Pierre Pham Van pas de disponibilité
Dr Hind Asri pas de disponibilité
Dr Sidonie Richard pas de disponibilité
Dr Philippe AMIEL pas de disponibilité
Dr Michel HARROCH pas de disponibilité
Dr Cannelle Moulines pas de disponibilité
Dr Benedicte Marill ['30', 'janvier']
Dr Alice LAPLACE pas de disponibilité
Dr Romain LANNEVAL ['19', 'janvier']
Dr Elodie Martinez ['23', 'janvier']
Dr Isabelle GLUCKLER pas de disponibilité
Mediksanté Toulouse jeu. 12 janv. 15:00
Dr André BENHAIM pas de disponibilité
Dr Joanna SIMOES ['23', 'janvier']
Dr Jean-Pierre Ginestet pas de disponibilité
Dr Benedicte Clairet Oustric ['30', 'janvier']
Centre de Santé Hygie - Toulouse ['16', 'janvier']
Dr Borhane FERJANI sam. 14 janv. 08

We can see that there are no disponibilites for any doctor. This means that we will have to dig for more doctors as we are only taking the first ones at the moment.

In [ ]:
#Now we need to find the fees for each doctor

connector = Driver()
connector.connect("https://www.doctolib.fr")

doctors = pd.read_csv('doctors.csv')
doctors.pop(doctors.columns[0])


for i in range(50):
    print(doctors['Name'][i])
    fee = connector.getFees(doctors['Name'][i])
    doctors['Fees'][i] = fee
    print(i, fee)


doctors.to_csv("doctors2.csv")
    

/var/folders/bm/t8hxv1ms6s30y8czfw0pmfk00000gn/T/ipykernel_2201/4246120342.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())


Connecting to https://www.doctolib.fr


KeyboardInterrupt: 